<a href="https://colab.research.google.com/github/gitsika119/ObjectClassification/blob/main/ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#uncomment line below to install ultralytics library before first run :)
#!pip install opencv-python ultralytics

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from google.colab import drive
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# Mounted Google Drive
drive.mount('/content/drive')

# Loaded yolo and resnet models
yolo_model = YOLO("yolov8n.pt")
resnet_model = ResNet50(weights='imagenet')

# Resnet functions
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array
def classify_image(img_path):
    img_array = load_and_preprocess_image(img_path)
    predictions = resnet_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions,1)[0]
    return decoded_predictions
def display_image_with_predictions(img_path, predictions):
    img = image.load_img(img_path)
    img_width, img_height = img.size
    border_thickness = 10
    blue_color = (255, 0, 0)
    bordered_img = cv2.copyMakeBorder(
        cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR),
        border_thickness, border_thickness, border_thickness, border_thickness,
        cv2.BORDER_CONSTANT, value=blue_color
    )
    pred_text = "\n".join([f"{label}" for (imagenet_id, label, score) in predictions])
    text_x = 300
    text_y = img_height - 30
    font_scale = 1
    for line in pred_text.split('\n'):
        (text_width, text_height), _ = cv2.getTextSize(line, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
        cv2.putText(bordered_img, line, (text_x, text_y),
                    cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 0, 0), 2, cv2.LINE_AA)
    bordered_img_rgb = cv2.cvtColor(bordered_img, cv2.COLOR_BGR2RGB)
    plt.imshow(bordered_img_rgb)
    plt.axis('off')
    plt.title(os.path.basename(img_path))
    plt.show()

# yolo functions
def predict_and_process_images(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        image = cv2.imread(img_path)
        if image is None:
            print(f"Error loading image: {img_path}")
            continue
        results = yolo_model.predict(image)
        if results and any(len(result.boxes) > 0 for result in results):
            occupied_positions = []
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    class_name = result.names[int(box.cls[0])]
                    confidence = box.conf[0]
                    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                    text = f"{class_name}: {confidence:.2f}"
                    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                    text_x = int(x1)
                    text_y = int(y1) - 10
                    if text_y < 0:
                        text_y = int(y1) + text_size[1] + 10
                    while any((text_x < x + w and text_x + text_size[0] > x and text_y < y + h and text_y + text_size[1] > y)
                               for (x, y, w, h) in occupied_positions):
                        text_y += text_size[1] + 5
                    occupied_positions.append((text_x, text_y, text_size[0], text_size[1]))
                    cv2.putText(image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()

        else:
            # no detections by yolo, use resnet functions
            predictions = classify_image(img_path)
            display_image_with_predictions(img_path, predictions)

#save folder of images in drive and change folder path if needed:
folder_path = "/content/drive/MyDrive/Images/Images"
predict_and_process_images(folder_path)
